In [2]:
import pymongo
import pandas as pd

import plotly.figure_factory as ff

from flaskapp import config

In [3]:
myclient = pymongo.MongoClient(str(config.MONGO_URI))
stagingDb = myclient[str(config.DB_STAGING)]

In [10]:
pipeline = [ 
            { '$group':{'_id': {"projectcd" : "$project", 'table': "$table"}}},
            { '$unwind': "$_id"},
            { '$project': {'_id': 0, 'projectcd': { '$ifNull': [ "$_id.projectcd", "Unspecified" ] }, 'table': "$_id.table"}},
            { '$addFields': {'type':'Alleen Data'}}
        ]
dfIngelezenTabellenOud = pd.DataFrame(list(stagingDb[config.COLL_STAGING_OUD].aggregate(pipeline)))
dfIngelezenTabellenNieuw = pd.DataFrame(list(stagingDb[config.COLL_STAGING_NIEUW].aggregate(pipeline)))
dfIngelezenTabellenDelfIT = pd.DataFrame(list(stagingDb[config.COLL_STAGING_DELFIT].aggregate(pipeline)))
dfIngelezenTabellenMagazijn = pd.DataFrame(list(stagingDb[config.COLL_STAGING_MAGAZIJNLIJST].aggregate(pipeline)))

dfIngelezenTabellen = result = pd.concat([dfIngelezenTabellenOud, dfIngelezenTabellenNieuw, dfIngelezenTabellenDelfIT, dfIngelezenTabellenMagazijn])

pipeline = [ 
            { '$group':{'_id': {"projectcd" : "$project", 'table': "$table"}}},
            { '$unwind': "$_id"},
            { '$project': {'_id': 0, 'projectcd': { '$ifNull': [ "$_id.projectcd", "Unspecified" ] }, 'table': "$_id.table"}},
            { '$addFields': {'type':'Alleen Meta'}}
        ]
ingelezenMetadata = stagingDb['Kolominformatie'].aggregate(pipeline)
dfIngelezenMetadata = pd.DataFrame(list(ingelezenMetadata))


In [11]:

df = pd.merge(dfIngelezenTabellen, dfIngelezenMetadata, how='outer', on=['projectcd', 'table'])
df['ingelezen'] = df.apply(lambda x: "Data en Meta" if pd.notna(x['type_y']) & pd.notna(x['type_x']) else 
    (x['type_x'] if pd.notna(x['type_x']) else
    (x['type_y'] if pd.notna(x['type_y']) 
    else "Niets")),axis=1)
df['teller'] = 1
df = df.sort_values(by='projectcd', ascending=True, na_position='last')

In [12]:
import plotly.express as px
fig = px.bar(df, x='ingelezen', y='teller', color='projectcd', hover_data=['table'])
fig.show()